[exercises](ir.ipynb)

In [1]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import soundfile as sf
import tools

Using matplotlib backend: TkAgg


In [2]:
sig, fs = sf.read('data/xmas.wav')
fs

44100

In [3]:
sig_blackbox = tools.blackbox(sig, fs)

In [4]:
sf.write('data/xmas_blackbox.wav', sig_blackbox, fs)

<audio src="data/xmas.wav" controls>Your browser does not support the audio element.</audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_blackbox.wav" controls>Your browser does not support the audio element.</audio>
[data/xmas_blackbox.wav](data/xmas_blackbox.wav)

The system is supposed to sound like narrow-band telephony (limited to a frequency range from 300 to 3400 Hz).

In [5]:
dur = 1/10  # seconds
imp = np.zeros(np.ceil(dur * fs))
imp[0] = 1

In [6]:
ir = tools.blackbox(imp, fs)

In [7]:
t = np.arange(len(imp)) / fs
plt.plot(t, ir)
plt.figure()
plt.plot(t, tools.db(ir));

In [8]:
w, h = signal.freqz(ir)
plt.figure()
plt.plot(w * fs / (2 * np.pi), tools.db(h))  # logarithmic y axis
#plt.plot(w * fs / (2 * np.pi), np.abs(h))  # linear y axis
plt.xscale('log')

In [9]:
import numpy as np

def naive_convolution(x, h):
    """Very inefficient convolution of two one-dimensional arrays."""
    totalsize = len(x) + len(h) - 1
    y = np.zeros(totalsize)  # Allocate and initialize memory
    for n in range(totalsize):
        # m must be within range(len(x)) and it must meet the condition
        # n >= m and (n-m) < len(h)
        m_start = max(n + 1 - len(h), 0)
        m_stop = min(n + 1, len(x))
        for m in range(m_start, m_stop):
            y[n] += x[m] * h[n - m]
    return y

In [10]:
# This takes a loooong time:
#sig_naive = naive_convolution(sig, ir)

# Let's just try a tiny bit of the signal:
%time sig_naive = naive_convolution(sig[:1000], ir)

CPU times: user 1.58 s, sys: 32 ms, total: 1.61 s
Wall time: 1.58 s


`len(y) == len(x) + len(h) - 1`

The result should obviously be the same (but with `blackbox()` the resulting signal is shorter because the end is cut off).

In [11]:
# TODO: matrix convolution

# TODO: use numpy.lib.stride_tricks.as_strided() for Toeplitz matrix construction:
# http://stackoverflow.com/a/21028494/500098

In [12]:
# TODO: fast convolution (rfft and fft?)

Yes, it should indeed be faster. *Very very much* faster!

In [13]:
%time sig_convolve = np.convolve(sig, ir)

CPU times: user 180 ms, sys: 4 ms, total: 184 ms
Wall time: 181 ms


In [14]:
%time sig_fftconvolve = signal.fftconvolve(sig, ir)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 20.4 ms


Note that with short durations, the result of `%time` isn't really reliable, because the measurement may be influenced by other things going on before and after the actual execution of the function and other processes executed by the operating system at the same time.

To get a more reliable estimation, use `%timeit` instead.

In [15]:
# TODO: longer signal for more realistic comparison

In [16]:
sig_blackbox_nonlinear = tools.blackbox_nonlinear(sig, fs)

In [17]:
sf.write('data/xmas_blackbox_nonlinear.wav', sig_blackbox_nonlinear, fs)

The non-linear system should sound like a distorted narrow-band telephone.

In [18]:
ir_nonlinear = tools.blackbox_nonlinear(imp, fs)

In [19]:
sig_nonlinear_convolve = np.convolve(sig, ir_nonlinear)

In [20]:
sf.write('data/xmas_convolution_nonlinear.wav', sig_nonlinear_convolve, fs)

<audio src="data/xmas.wav" controls>Your browser does not support the audio element.</audio>
[original](data/xmas.wav)

<audio src="data/xmas_blackbox.wav" controls>Your browser does not support the audio element.</audio>
[through `tools.blackbox()`](data/xmas_blackbox.wav)

<audio src="data/xmas_blackbox_nonlinear.wav" controls>Your browser does not support the audio element.</audio>
[through `tools.blackbox_nonlinear()`](data/xmas_blackbox_nonlinear.wav)

<audio src="data/xmas_convolution_nonlinear.wav" controls>Your browser does not support the audio element.</audio>
[trying to use the impulse response of the non-linear system (doesn't work!)](data/xmas_convolution_nonlinear.wav)



<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>